In [1]:
import re
import numpy as np
import scipy
import itertools
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from collections import Counter
from random import choice
import pandas as pd

data = pd.read_csv('/')

In [2]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-cdada76f-7bbb-bc43-3ee5-4d71f7613996)


In [3]:
data.shape

(159571, 8)

In [4]:
#preprocess
import os
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import  stopwords
import string
import re
    
data['length'] = data['comment_text'].str.len()

# Convert all messages to lower case
data['comment_text'] = data['comment_text'].str.lower()


# Replace email addresses with 'email'
data['comment_text'] = data['comment_text'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress')

# Replace URLs with 'webaddress'
data['comment_text'] = data['comment_text'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
data['comment_text'] = data['comment_text'].str.replace(r'£|\$', 'dollars')

    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
data['comment_text'] = data['comment_text'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumber')


# Replace numbers with 'numbr'
data['comment_text'] = data['comment_text'].str.replace(r'\d+(\.\d+)?', 'numbr')




data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join(
    term for term in x.split() if term not in string.punctuation))


stop_words = set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])
data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))


lem=WordNetLemmatizer()
data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join(
 lem.lemmatize(t) for t in x.split()))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: The default value of regex will change from True to False in a future version.


In [5]:
#tokenization
tokenized_text = data['comment_text'].apply(lambda x: x.split())

tokenized_text.head()

0    [explanation, edits, made, username, hardcore,...
1    [d'aww!, match, background, colour, i'm, seemi...
2    [hey, man,, i'm, really, trying, edit, war., g...
3    [can't, make, real, suggestion, improvement, w...
4    [you,, sir,, hero., chance, remember, page, th...
Name: comment_text, dtype: object

In [6]:
data['clean_length'] = data.comment_text.str.len()
data.head(3)

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,length,clean_length
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0,264,180
1,000103f0d9cfb60f,d'aww! match background colour i'm seemingly s...,0,0,0,0,0,0,112,111
2,000113f07ec002fd,"hey man, i'm really trying edit war. guy const...",0,0,0,0,0,0,233,149


In [8]:
#splitting data into test&train
train = data.sample(frac=0.8)
test = data.sample(frac=0.2)

In [9]:
#Term Frequency-Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')

train_tfidf_matrix=tfidf.fit_transform(train['comment_text'])
test_tfidf_matrix=tfidf.fit_transform(test['comment_text'])

train_df_tfidf = pd.DataFrame(train_tfidf_matrix.todense())
test_df_tfidf = pd.DataFrame(test_tfidf_matrix.todense())

In [10]:
train_df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127652,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127653,0.436417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127654,0.261813,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127655,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.226214,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
train_tfidf_matrix.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.26181339, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [12]:
comments=np.array(train.iloc[:,1])
malignant=np.array(train.iloc[:,2])
highly_malignant=np.array(train.iloc[:,3])
rude = np.array(train.iloc[:,4])
threat=np.array(train.iloc[:,5])
abuse=np.array(train.iloc[:,6])
loathe=np.array(train.iloc[:,7])

In [33]:
training_matrix = train_tfidf_matrix.todense()
print(training_matrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.26181339 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [34]:
#splitting the data into train and validation

from sklearn.model_selection import train_test_split

#TF-IDF features

x_train, x_valid, y_train, y_valid = train_test_split(training_matrix,
                                                                              train.iloc[:,2:8],
                                                                              test_size=0.3)

In [35]:
print(x_train)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.22471065 0.         0.51823003 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier(criterion='entropy')
dct.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)

In [ ]:
#predicting the probabilities
dct_prediction = dct.predict_proba(x_valid)
knn_prediction = knn.predict_proba(x_valid)

In [83]:
dct_prediction

[array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]])]

In [40]:
dct_malignant = dct_prediction[0]
dct_highly_malignant = dct_prediction[1]
dct_rude = dct_prediction[2]
dct_threat = dct_prediction[3]
dct_abuse = dct_prediction[4]
dct_loathe = dct_prediction[5]

In [42]:
y_valid

,malignant,highly_malignant,rude,threat,abuse,loathe
118259,0,0,0,0,0,0
23544,0,0,0,0,0,0
21583,0,0,0,0,0,0
102905,0,0,0,0,0,0
1590,0,0,0,0,0,0
...,...,...,...,...,...,...
59527,0,0,0,0,0,0
156938,0,0,0,0,0,0
128530,0,0,0,0,0,0
3700,0,0,0,0,0,0


In [43]:
malignant_valid = np.array(y_valid.iloc[:,0])
highly_malignant_valid = np.array(y_valid.iloc[:,1])
rude_valid = np.array(y_valid.iloc[:,2])
threat_valid = np.array(y_valid.iloc[:,3])
abuse_valid = np.array(y_valid.iloc[:,4])
loathe_valid = np.array(y_valid.iloc[:,5])


In [44]:
malignant_valid

array([0, 0, 0, ..., 0, 0, 0])

In [72]:
from sklearn.metrics import mean_squared_error as mse

dct_prediction_malignant = dct_malignant

# converting the results to integer type
dct_prediction_malignant_int = dct_prediction_malignant.astype(np.int)
dct_prediction_malignant_int

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [85]:
dct_score_mse= mse(y_valid,dct_prediction)

dct_score_mse


[[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]


ValueError: ignored

In [ ]:
a